# Decision Trees Netflix

In [18]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

from sklearn.tree import DecisionTreeClassifier 

from sklearn.metrics import classification_report

from sklearn.tree import plot_tree

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [19]:
Netflix = pd.read_csv("/Users/jackbrennan/Documents/GitHub/Stock-Predictions/Alex/Netflix_model_ready.csv")
Netflix.date = pd.to_datetime(Netflix.date)
Netflix = Netflix.set_index("date")
Netflix = Netflix.drop(["Nas_total", 'Stock_total', 'Dow_total'],axis=1) # to remove duplicated columns

Netflix = Netflix.dropna()
Netflix = Netflix[~(Netflix.isin([np.inf, -np.inf]).any(axis=1))] 

Netflix = Netflix.drop(['target_1', 'target_2', 'target_4', 'target_5'], axis=1)
target_3 = Netflix["target_3"]

In [20]:
#splitting into training sets 
column_indices = {name: i for i, name in enumerate(Netflix.columns)}

n = len(Netflix)
X_train = Netflix[0:int(n*0.7)]
X_test = Netflix[int(n*0.7):]

y_train = target_3[0:int(n*0.7)]
y_test = target_3[int(n*0.7):]

In [21]:
Mscaler = MinMaxScaler() # keeps binarys at zero and 1 :)

X_train = pd.DataFrame(Mscaler.fit_transform(X_train), columns = Netflix.columns)
X_test = pd.DataFrame(Mscaler.fit_transform(X_test), columns = Netflix.columns)

In [22]:
def kbest_creator(k, df, df_test):
    """
    returns data frame of principle componets of # of k best features 
    """
    # apply SelectKBest class to extract top 40 best features
    bestfeatures = SelectKBest(score_func=f_regression, k=k)
    best_fit = bestfeatures.fit(df, y_train)
    best_scores = pd.DataFrame(best_fit.scores_)
    best_columns = pd.DataFrame(Netflix.columns)
    
    # concatenate the dataframes for better visualization
    features_score = pd.concat([best_columns, best_scores], axis=1)
    features_score.columns = ['Features', 'Score']  # naming the dataframe columns
    feats_kb = list(features_score.nlargest(k, 'Score').iloc[1:k]['Features'])

    pca = PCA().fit(df[feats_kb])
    pca_scores = np.cumsum(pca.explained_variance_ratio_) * 100
    res = next(x for x, val in enumerate(pca_scores) if val > 85)
    res = res + 1
    res
    
    pca_kb_1 = PCA(n_components = res).fit(df[feats_kb].to_numpy())
    df = pca_kb_1.transform(df[feats_kb].to_numpy())
    df_test = pca_kb_1.transform(df_test[feats_kb].to_numpy())
    return df, df_test

In [23]:
X_train_kb_10, X_test_kb_10 = kbest_creator(11, X_train, X_test) 
X_train_kb_25, X_test_kb_25 = kbest_creator(26, X_train, X_test) 
X_train_kb_40, X_test_kb_40 = kbest_creator(41, X_train, X_test) 
X_train_kb_55, X_test_kb_55 = kbest_creator(56, X_train, X_test) 
X_train_kb_75, X_test_kb_55 = kbest_creator(76, X_train, X_test) 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/fe

In [24]:
counts = y_train.groupby(by = y_train).count()
counts = pd.Series(counts)
print(counts)

target_3
0    244
1    260
Name: target_3, dtype: int64


# Model Creating

In [25]:
# Model1, 10 k best variables with PCA
random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': list(np.linspace(10, 1200, 10, dtype = int)) + [None],
               'max_features': ['auto', 'sqrt','log2', None],
               'min_samples_leaf': [4, 6, 8, 12],
               'min_samples_split': [5, 7, 10, 14],
               'n_estimators': list(np.linspace(10, 1200, 15, dtype = int))}

clf = RandomForestClassifier()
model1 = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 100, 
                               cv = 10, verbose= 5, random_state= 101, n_jobs = -1)
model1.fit(X_train_kb_10,y_train)

y_kb_hat_10 = model1.best_estimator_.predict(X_test_kb_10)
accuracy_score(y_test, y_kb_hat_10)


Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 2/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.451 total time=   2.6s
[CV 4/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.471 total time=   2.6s
[CV 3/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.510 total time=   2.7s
[CV 1/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.510 total time=   2.7s
[CV 5/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.500 total time=   2.6s
[CV 6/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.580 total time=   2.7s

0.5391705069124424

In [26]:
# Model1, 25 k best variables with PCA
random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': list(np.linspace(10, 1200, 10, dtype = int)) + [None],
               'max_features': ['auto', 'sqrt','log2', None],
               'min_samples_leaf': [4, 6, 8, 12],
               'min_samples_split': [5, 7, 10, 14],
               'n_estimators': list(np.linspace(10, 1200, 15, dtype = int))}

clf = RandomForestClassifier()
model2 = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 100, 
                               cv = 10, verbose= 5, random_state= 101, n_jobs = -1)
model2.fit(X_train_kb_25,y_train)

y_kb_hat_25 = model2.best_estimator_.predict(X_test_kb_25)
accuracy_score(y_test, y_kb_hat_25)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 2/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.431 total time=   3.4s
[CV 1/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.412 total time=   3.4s
[CV 4/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.412 total time=   3.4s
[CV 3/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.412 total time=   3.4s
[CV 6/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.660 total time=   3.5s
[CV 5/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.500 total time=   3.6s

0.5161290322580645

In [27]:
# Model3, 40 k best variables with PCA
random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': list(np.linspace(10, 1200, 10, dtype = int)) + [None],
               'max_features': ['auto', 'sqrt','log2', None],
               'min_samples_leaf': [4, 6, 8, 12],
               'min_samples_split': [5, 7, 10, 14],
               'n_estimators': list(np.linspace(10, 1200, 15, dtype = int))}

clf = RandomForestClassifier()
model3 = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 100, 
                               cv = 10, verbose= 5, random_state= 101, n_jobs = -1)
model3.fit(X_train_kb_40,y_train)

y_kb_hat_40 = model3.best_estimator_.predict(X_test_kb_40)
accuracy_score(y_test, y_kb_hat_40)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 1/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.510 total time=   3.3s
[CV 3/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.471 total time=   3.4s
[CV 4/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.431 total time=   3.4s
[CV 2/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.471 total time=   3.4s
[CV 5/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.500 total time=   3.6s
[CV 8/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.560 total time=   3.5s

0.4792626728110599

In [28]:
# Model4, 75 k best variables with PCA
random_search = {'criterion': ['entropy', 'gini'],
               'max_depth': list(np.linspace(10, 1200, 10, dtype = int)) + [None],
               'max_features': ['auto', 'sqrt','log2', None],
               'min_samples_leaf': [4, 6, 8, 12],
               'min_samples_split': [5, 7, 10, 14],
               'n_estimators': list(np.linspace(10, 1200, 15, dtype = int))}

clf = RandomForestClassifier()
model4 = RandomizedSearchCV(estimator = clf, param_distributions = random_search, n_iter = 100, 
                               cv = 10, verbose= 5, random_state= 101, n_jobs = -1)
model4.fit(X_train_kb_75,y_train)

y_kb_hat_75 = model4.best_estimator_.predict(X_test_kb_55) 
accuracy_score(y_test, y_kb_hat_75)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV 4/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.451 total time=   3.0s
[CV 3/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.569 total time=   3.0s
[CV 1/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.471 total time=   3.1s
[CV 2/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.471 total time=   3.1s
[CV 5/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.520 total time=   3.2s
[CV 8/10] END criterion=gini, max_depth=274, max_features=log2, min_samples_leaf=12, min_samples_split=5, n_estimators=945;, score=0.420 total time=   3.2s

0.4976958525345622